# Dynamical System Approximation
This notebook aims at learning a functional correlation based on given snapshots. The data is created through the following ODE which is called the Fermi Pasta model:
\begin{align}
\frac{d^2}{dt^2} x_i = (x_{i+1} - 2x_i + x_{i-1}) + 0.7((x_{i+1} - x_i)^3 - (x_i-x_{i-1})^3) + \mathcal{N}(0,\sigma)
\end{align}



In [1]:
import numpy as np
import xerus
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import time 
from itertools import chain
import helpers as hp
import pandas as pd

%precision  4

'%.4f'

In [6]:
#Construction of the exact solution in the choosen basis.
#Transform implements the basis transformation for given b
#For the exact solution we also factor out the kernel, by use of the Pseudoinverse.
def transform(X):
    M = np.zeros([4,4])
    M[0,0] = 1   #1
    M[1,1] = 1   #1
    M[2,2] = 1.5 #1.5
    M[3,3] = 2.5 #2.5
    M[0,2] = -0.5 #-0.5
    M[1,3] = -1.5 #-1.5
    t = xerus.Tensor.from_ndarray(np.linalg.inv(M))
    a1,a2,a3,a4,b1,b2,b3,b4 = xerus.indices(8)
    for eq in range(noo):
        tmp = X.get_component(eq)
        tmp2 = C2list[eq]
        tmp(a1,a2,a3,a4) <<  tmp(a1,b2,a3,a4)* t(a2,b2) 
        X.set_component(eq,tmp)
    return X

def project(X):
    dim = ([(3 if i == 0 or i == noo -1 else 4) for i in range(0,noo)])
    dim.extend(dim)
    C2T = xerus.TTOperator(dim)    
    for eq in range(noo):
        idx = [0 for i in range(noo)]
        if eq == 0:
            idx[0] = 2
            idx[1] = 3
        elif eq == noo -1:
            idx[noo-2] = 1
            idx[noo-1] = 1
        elif eq == noo -2:
            idx[eq-1] = 1
            idx[eq]   = 2
            idx[eq+1] = 2
        else:
            idx[eq-1] = 1
            idx[eq]   = 2
            idx[eq+1] = 3
        idx.extend(idx)
        C2T += xerus.TTOperator.dirac(dim,idx) 
    C2T.round(1e-12)
    i1,i2,i3,i4,i5,i6,j1,j2,j3,j4,k1,k2,k3 = xerus.indices(13)

    X(i1^noo,j1^noo) << X(i1^noo,k1^noo) * C2T(k1^noo,j1^noo)
    X.round(1e-12)
    return X

def exact(noo,p):
    beta = 0.7
    C1ex = hp.construct_exact_fermit_pasta(noo,p,beta)
    C1ex = transform(C1ex)  
    C1ex = project(C1ex) 
    return C1ex


### Recovery algorithm
We want to recover the exact solution with the help of a regularized ALS.

In [7]:
# initialize simulation
def initialize(p,noo):
    rank = 4 #fix rank
    dim = [p for i in range(0,noo)]
    dim.extend([4 for i in range(0,noo)])
    dim[noo] = 3
    dim[2*noo-1]=3
    C = xerus.TTOperator.random(dim,[rank for i in range(0,noo-1)]) # initalize randomly
    C.move_core(0,True)
    return C

In [8]:
# We choose different pairstriples  of dimensions samplesizes and noise level to run the algoirthm for.
data_noo_nos = [(12,4500,1e-8),(12,4500,1e-7),(12,4500,1e-6),(12,4500,1e-5),(12,4500,1e-4)\
                ,(12,4500,1e-3),(12,4500,1e-2),(12,4500,1e-1),(12,4500,1)] 
#                                                       pairs used in simulations in the paper
#                                                       uncomment to simulate but is computational intensive
data_noo_nos = [(12,4500,1e-2),(12,4500,1e-1),(12,4500,1)] 
#data_noo_nos = [(8,3000)] #specify pairs to simulate for
#runs = 1 #specify number of runs for each pair (10 in the paper)
runs = 3
max_iter = 30 # specify number of sweeps
output = 'data.csv' # specify name of output file

# build data structure to store solution
tuples = []
for data in data_noo_nos:
    noo = data[0]
    nos = data[1] 
    sigma = data[2]
    for r in range(0,runs):
        tuples.append((noo,nos,sigma, r))
index = pd.MultiIndex.from_tuples(tuples, names=['d', 'm','sigma','runs'])           

# The results of each optimization is store in a Dataframe
col = ["data norm", "noise norm"]
col.extend([i for i in range(1,max_iter+1)])
df = pd.DataFrame(np.zeros([len(tuples),max_iter+2]), index=index,columns=col) 
print(len(index))
df["data norm"]

9


d   m     sigma  runs
12  4500  0.01   0       0.0
                 1       0.0
                 2       0.0
          0.10   0       0.0
                 1       0.0
                 2       0.0
          1.00   0       0.0
                 1       0.0
                 2       0.0
Name: data norm, dtype: float64

In [ ]:
np.set_printoptions(precision=4)
#loop over all pairs of samples, calls hp.run_als for the solution
lam = 1 #regularization parameter
#Master iteration
psi = hp.basis(0) # get basis functions, Legendre
p = len(psi)
for data in data_noo_nos:
    noo = data[0]
    nos = data[1]
    sigma = data[2]
    print( "(noo,nos,sigma) = (" + str(noo) +',' + str(nos) +',' + str(sigma) + ')' )
    C2list = hp.build_choice_tensor2(noo) # build selection tensor as list of pxnos matrices
    C1ex = exact(noo,p) # construct exact solution
    print("C1ex frob_norm: " +str(C1ex.frob_norm()))
    for r in range(runs):
        [x, y] = hp.fermi_pasta_ulam(noo, nos) # create samples and labels
        df["data norm"].loc[(noo,nos,sigma,r)] = np.linalg.norm(y)
        noise = np.random.normal(0,sigma,size=y.shape)
        df["noise norm"].loc[(noo,nos,sigma,r)] = np.linalg.norm(noise)
        y = y + noise
        Alist = hp.build_data_tensor_list2(noo,x,nos,psi,p) # build the dictionary tensor for the given samples x
        Y = xerus.Tensor.from_ndarray(y)
        C1 = initialize(p,noo) # initialize the als randomly
        errors = hp.run_als(noo,nos,C1,C2list,Alist,C1ex,Y,max_iter,lam) # run the regularized ALS iteration
        #post processing, store data in dataframe
        for i in range(1,len(errors)):
            df[i].loc[(noo,nos,sigma,r)] = errors[i-1]

        print("Run: " +str(r) + " finished result = " + str(errors) + " data norm = " + str( df["data norm"].loc[(noo,nos,sigma,r)])+ " noise norm = " + str( df["noise norm"].loc[(noo,nos,sigma,r)]))
        df.to_csv(output)

L2(-1,1) orthogonal polynomials
(noo,nos,sigma) = (12,4500,0.01)
C1ex frob_norm: 19.885773809434728
